In [115]:
from subprocess import check_output

In [116]:
traindatafile = '../input/fashion-mnist_train.csv'
testdatafile = '../input/fashion-mnist_test.csv'

In [117]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [118]:
MODEL_DIR = './fashion-model'

In [119]:
features = [tf.feature_column.numeric_column('pixels', shape=[28,28])]

In [120]:
classifier = tf.estimator.LinearClassifier(
    feature_columns=features,
    n_classes=10,
    model_dir=MODEL_DIR
)

In [121]:
def generate_labelled_input_fn(csv_files, batch_size):
    def input_fn():
        file_queue = tf.train.string_input_producer(csv_files)
        reader = tf.TextLineReader(skip_header_lines=1)
        _, rows = reader.read_up_to(file_queue, num_records=100*batch_size)
        expanded_rows = tf.expand_dims(rows, axis=-1)
        shuffled_rows = tf.train.shuffle_batch(
            [expanded_rows],
            batch_size=batch_size,
            capacity=20*batch_size,
            min_after_dequeue=5*batch_size,
            enqueue_many=True
        )
        record_defaults = [[0] for _ in range(28*28+1)]
        columns = tf.decode_csv(shuffled_rows, record_defaults=record_defaults)
        labels = columns[0]
        pixels = tf.concat(columns[1:], axis=1)
        return {'pixels': pixels}, labels
    return input_fn

In [122]:
BATCH_SIZE = 40

In [123]:
TRAIN_STEPS = 2000

In [124]:
classifier.train(
    input_fn=generate_labelled_input_fn([traindatafile], BATCH_SIZE),
    steps=TRAIN_STEPS
)

In [125]:
classifier.evaluate(
    input_fn=generate_labelled_input_fn([testdatafile], BATCH_SIZE),
    steps=100
)

{'accuracy': 0.8315,
 'average_loss': 27.045015,
 'loss': 1081.8007,
 'global_step': 12000}

In [126]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [136]:
CLASSES = {
    '0': 'Tees',
    '1': 'Trousers',
    '2': 'Sweat shirt',
    '3': 'Gown',
    '4': 'Over Coat',
    '5': 'Sandal',
    '6': 'Shirt',
    '7': 'Shoes',
    '8': 'Bag',
    '9': 'Boots'
}

In [137]:
test_data = pd.read_csv(testdatafile)

In [138]:
sample_row = test_data.sample()

In [139]:
sample = list(sample_row.iloc[0])
label = sample[0]
pixels = sample[1:]

In [140]:
image_array = np.asarray(pixels, dtype=np.float32).reshape((28, 28))

In [141]:
def generate_prediction_input_fn(image_arrays):
    def input_fn():
        queue = tf.train.input_producer(
            tf.constant(np.asarray(image_arrays)),
            num_epochs=1
        )
        
        image = queue.dequeue()
        return {'pixels': [image]}
    
    return input_fn

In [142]:
predictions = classifier.predict(
    generate_prediction_input_fn([image_array])
)

In [143]:
prediction = next(predictions)

In [144]:
print('Actual label: {}'.format(CLASSES[str(label)]))
predicted_class = prediction['classes'][0].decode('utf-8')
print('Predicted class: {}'.format(
    CLASSES[predicted_class]
))

Actual label: Boots
Predicted class: Boots
